### Quant Exercise

**1. Load in NMR Data between Sept 2010 and July 2011**
- Create a table from the nmrSept2010July2011 csv file located in the data folder
- Add a column called sym with a value `NMR
- Rename the columns so that they are lower case, remove any spaces and replace with underscores

In [ ]:
nmr1:("DFFFFFI"; enlist ",") 0: `$":./data/nmrSept2010July2011.csv"
nmr1:update sym:`NMR from `date`open`high`low`close`adj_close`volume xcol nmr1
2#nmr1

**2. Load in NMR Data between July 2011 and July 2012**
- Create a table from the nmrJuly2011July2012 csv file located in the data folder
- Add a column called sym with a value `NMR
- Rename the columns so that they are lower case, remove any spaces and replace with underscores
- Join the newly created table with the table in task 1
- Remove duplicate rows & reorder the table by date
- Remove any data points where the daily volume is less than 5% of the daily average

In [ ]:
nmr2:("DFFFFFI"; enlist ",") 0: `$":./data/nmrJuly2011July2012.csv"
nmr2:update sym:`NMR from `date`open`high`low`close`adj_close`volume xcol nmr2
nmr:`date xdesc distinct nmr1,nmr2
2#nmr

In [ ]:
// creating avg_date_function, used to calculate daily average

//parse "exec volume from tab where date within x"
//(sum input list) % count of the input list

// x --> table
// y --> date window
// z --> column you are extracting

//extract the list where date is within the date window and take the avg of it
avg_date_function:{[x;y;z] 
    avg ?[x;enlist (within;`date;y);();z] }

//date window to pass through (22 days in this case)
date_windows:((nmr`date) - 22),'(nmr`date)

//Creating an avgDaily table
nmr:update avg_daily:avg_date_function[nmr;;`volume] each date_windows from nmr

//Removing all rows with a volume less than 5% of the average daily volume
nmr:select from nmr where volume > 0.05*avg_daily
2#nmr

**3. Create a new table for NDX data**
- Create a table from the ndrSept2010July2012 csv file located in the data folder
- Add a column called sym_rff with a value `NDX
- Rename the columns so that they are lower case, remove any spaces and replace with underscores and add a trailing _rff (except to date)

In [ ]:
ndx:("DFFFFFI"; enlist ",") 0: `$":./data/ndrSept2010July2012.csv"
ndx:update sym_rff:`NDX from `date`open_rff`high_rff`low_rff`close_rff`adj_close_rff`volume_rff xcol ndx
2#ndx

**4. Join together the completed ndx and nmr tables on the key date**
- Join the newly created table using the `lj` command
- Fill any existing data points with the previous non-null values

In [ ]:
//Creating a new table using left join
joined_tab:nmr lj `date xkey ndx

//fill fowarding each column in the table
parse "update volume_rff:fills volume_rff from ndx1";
joined_tab:{[x;y] ![x;();0b;(enlist y)!enlist (^\;y)]}/[joined_tab;cols joined_tab]
2#joined_tab

**5. Create new columns to get the daily retuns for each NMR and NDX**
- Calculate the daily return for each stock using the formula: 
    `100 * (close price - yesterdays close price)/yesterdays close price`

In [ ]:
joined_tab:update 
    daily_ret_nrm:100*(close - next close)%next close, 
    daily_ret_ndx:100*(close_rff - next close_rff)%close_rff from joined_tab


2#select date, close, daily_ret_nrm from joined_tab 

**6. Calculate the average daily return and the standard deviation of the daily returns for each symbol**
- Use a 22 day work window for both values

In [ ]:
//average daily return from rest for 22 days
dates_window:((joined_tab`date) - 22),'(joined_tab`date)

//adding the average daily column to the table
joined_tab:update 
    avg_daily_ret_nrm:avg_date_function[joined_tab;;`daily_ret_nrm] each dates_window, 
    avg_daily_ret_ndx:avg_date_function[joined_tab;;`daily_ret_ndx] each dates_window from joined_tab

//Standard Deviation of the daily average rolling back from the previous 22 days from the input date
std_func:{[x;y;z] dev ?[x;enlist (within;`date;y);();z]}

joined_tab:update 
    std_nmr:std_func[joined_tab;;`daily_ret_nrm] each dates_window, 
    std_ndx:std_func[joined_tab;;`daily_ret_ndx] each dates_window from joined_tab

**7. Calculate a moving 5 day average for each symbol**

In [ ]:
//5 day window for 5 day moving average
dates_window:((joined_tab`date) - 5),'(joined_tab`date)

joined_tab:update m5d_avg_nmr:avg_date_function[joined_tab;;`close] each dates_window,
    m5d_avg_ndx:avg_date_function[joined_tab;;`close_rff] each dates_window from joined_tab

meta joined_tab

**8. Calculate the Beta and Alpha of NMR compared to NDX**
- Using a 273 work window to calculate the beta i.e. using a date window of 2011.06.01 to 2010.09.01

In [ ]:
//273 day work window for calculating the beta
dates_window:((joined_tab`date) - 273),'(joined_tab`date)

//covariance function for each date_window
cov_date_func:{[t;dw;col1;col2]
    (?[t;enlist (within;`date;dw);();col1]) cov ?[t;enlist (within;`date;dw);();col2]}

var_date_func:{[t;dw;col1]
    var ?[t;enlist (within;`date;dw);();col1]}

//creating the coveriance column between the two daily returns of both nmr and ndx with 273 day windows
joined_tab:update 
    covariance:cov_date_func[joined_tab;;`daily_ret_nrm;`daily_ret_ndx] each dates_window,
    variance_ndx:var_date_func[joined_tab;;`daily_ret_ndx] each dates_window from joined_tab

//Calculating the beta of the stock using the formula:
// beta: (covariance(nmr_daily_ret;ndx_daily_ret)/variance(ndx_daily_return))
joined_tab:update beta:covariance%(variance_ndx) from joined_tab 

select date, sym, sym_rff, covariance, variance_ndx, beta from joined_tab where date in 2011.06.02 2011.06.01

**9. Save both of these files to the data folder as a csv**

In [ ]:
save `:data/joined_tab.csv

**10. Download the newly created csv to your local machine**
- create a scatterplot of NMR daily returns versus NDX daily returns
- Use a date range from 2011.06.01 to 2010.09.01 to create this scatterplot
- Add a line of best fit
- What is the beta for this date range, how does it compare to the beta calculated from point 8?